In [1]:
! pip install datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/d7/84/0df6c5981f5fc722381662ff8cfbdf8aad64bec875f75d80b55bfef394ce/datasets-3.2.0-py3-none-any.whl.metadata
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/89/ec/00d68c4ddfedfe64159999e5f8a98fb8442729a63e2077eb9dcd89623d27/filelock-3.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for dill<0.3.9,>=0.3.0 from https://files.pythonhosted.org/packages/c9/7a/cef76fd8438a42f96db64ddaa85280485a9c395e7df3db8158cfec1eee34/dill-0.3.8-py3-none-any.whl.metadata
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Obtaining dependency information for requests>=2.32.2 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Using cached requests-2.32.3-py3-none-any.whl.metadata


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from datasets import load_dataset
ds = load_dataset(data_dir="dataset", path="dataset")
example = ds['train'][0]

EmptyDatasetError: The directory at F:/GitHub/table-parser/yolo/dataset/dataset doesn't contain any data files

In [7]:
import json
import os
from tqdm import tqdm

# Путь к вашему файлу в COCO-формате
COCO_JSON = r"table_dataset/images/train/_annotations.coco.json"
# Папка, где лежат сами изображения
IMAGES_DIR = r"table_dataset/images/train"  
# Папка, куда мы хотим сложить новые txt-файлы
OUTPUT_LABELS_DIR = r"table_dataset/labels/train"

# Убедитесь, что папка для labels существует
os.makedirs(OUTPUT_LABELS_DIR, exist_ok=True)

with open(COCO_JSON, 'r', encoding='utf-8') as f:
    coco_data = json.load(f)

images_info = {img['id']: img for img in coco_data['images']}
categories_info = {cat['id']: cat['name'] for cat in coco_data['categories']}

# Пример: если у вас 2 класса: {"id":0,"name":"bordered"}, {"id":1,"name":"borderless"},
# то в YOLO labels будут: class 0 -> "bordered", class 1 -> "borderless"
# Важно, чтобы COCO category_id точно соответствовал индексам YOLO.

# Группируем аннотации по ID картинки
annotations_by_image = {}
for ann in coco_data['annotations']:
    img_id = ann['image_id']
    if img_id not in annotations_by_image:
        annotations_by_image[img_id] = []
    annotations_by_image[img_id].append(ann)

for img_id, image_dict in tqdm(images_info.items()):
    file_name = image_dict['file_name']
    width = image_dict['width']
    height = image_dict['height']
    
    # Имя txt-файла такое же, как у изображения, только с .txt
    # Например, image.jpg -> image.txt
    # (Аккуратнее, если у вас сложные пути, лучше брать os.path.splitext(file_name)[0])
    txt_file_name = os.path.splitext(file_name)[0] + ".txt"
    txt_path = os.path.join(OUTPUT_LABELS_DIR, txt_file_name)
    
    # Составляем контент для этого txt
    yolo_lines = []
    if img_id in annotations_by_image:
        for ann in annotations_by_image[img_id]:
            cat_id = ann['category_id']  # ID класса из COCO
            # COCO bbox: [x_min, y_min, width, height] (в пикселях)
            x, y, w, h = ann['bbox']
            
            # Координаты центра и ширины/высоты в YOLO-формате (нормализованные)
            x_center = x + w/2
            y_center = y + h/2
            x_center_norm = x_center / width
            y_center_norm = y_center / height
            w_norm = w / width
            h_norm = h / height

            # Формируем строчку: <class_id> <x_center> <y_center> <width> <height>
            # Все значения - float, YOLO читает в том числе с плавающей точкой
            yolo_line = f"{cat_id} {x_center_norm:.6f} {y_center_norm:.6f} {w_norm:.6f} {h_norm:.6f}"
            yolo_lines.append(yolo_line)
    
    # Записываем содержимое в txt файл
    with open(txt_path, "w", encoding="utf-8") as f_txt:
        for line in yolo_lines:
            f_txt.write(line + "\n")


100%|██████████| 238/238 [00:00<00:00, 7694.78it/s]


In [8]:
! pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/22/4a/5b2a7577c7b770aa5c232576fa4116f6c0527b745d2a04bf5a19bcfc80ca/ultralytics-8.3.72-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python>=4.6.0 from https://files.pythonhosted.org/packages/a4/7d/f1c30a92854540bf789e9cd5dde7ef49bbe63f855b85a2e6b3db8135c591/opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.4.1 from https://files.pythonhosted.org/packages/af/25/caa430865749d504271757cafd24066d596217e83326155993980bc22f97/scipy-1.15.1-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     -------------------------------- ----- 51.2/60.8 kB 871.5 kB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/11/c5/2370d96b31eb1841


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from ultralytics import YOLO

# Загружаем предобученные веса (например, yolo8s.pt)
model = YOLO('yolov8s.pt')

# Запускаем обучение
model.train(
    data='table_data.yaml',  # наш data.yaml
    epochs=50,               # кол-во эпох (меняйте под нужды)
    imgsz=640,               # размер входного изображения (можно увеличить до 640, 768, ...)
    batch=16,                # размер batch (корректируйте в зависимости от GPU)
    name='table_detection'   # подпапка для логов (runs/detect/table_detection/)
)

Ultralytics 8.3.72  Python-3.11.7 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i7-12700K)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=table_data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=table_detection4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

train: Scanning F:\GitHub\datasets\table_dataset\labels\train... 238 images, 0 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<00:00, 676.06it/s]

train: New cache created: F:\GitHub\datasets\table_dataset\labels\train.cache



val: Scanning F:\GitHub\datasets\table_dataset\labels\val... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<00:00, 750.92it/s]

val: New cache created: F:\GitHub\datasets\table_dataset\labels\val.cache


...

                   all         70        109      0.889       0.81      0.929       0.86



50 epochs completed in 1.362 hours.
Optimizer stripped from f:\GitHub\table-parser\runs\detect\table_detection4\weights\last.pt, 22.5MB
Optimizer stripped from f:\GitHub\table-parser\runs\detect\table_detection4\weights\best.pt, 22.5MB

Validating f:\GitHub\table-parser\runs\detect\table_detection4\weights\best.pt...
Ultralytics 8.3.72  Python-3.11.7 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i7-12700K)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:08<00:00,  2.77s/it]


                   all         70        109      0.933      0.814      0.933      0.873
              bordered         18         23          1      0.779      0.957      0.926
            borderless         52         86      0.866      0.849      0.908       0.82
Speed: 1.5ms preprocess, 112.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to f:\GitHub\table-parser\runs\detect\table_detection4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001B63750F510>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0